In [3]:
!pip install segmentation-models-pytorch -q

import os
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import segmentation_models_pytorch as smp
from sklearn.model_selection import train_test_split

# --- 2. CONFIGURATION ---
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 16          
EPOCHS = 15              
LR = 0.0001              
IMG_SIZE = (256, 256)    

# HARDCODED PATHS (Adjust if needed based on your specific input folder)
TRAIN_RGB_DIR = '/kaggle/input/depth-estimation-challenge/dataset_ready_for_kaggle/train/images'
TRAIN_DEPTH_DIR = '/kaggle/input/depth-estimation-challenge/dataset_ready_for_kaggle/train/depth'
TEST_RGB_DIR = '/kaggle/input/depth-estimation-challenge/dataset_ready_for_kaggle/test/images'
OUTPUT_DIR = '/kaggle/working/predictions'
SUBMISSION_FILE = 'submission.csv'

os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Running on device: {DEVICE}")

# --- 3. HELPER: COMPETITION METRIC LOGIC ---
# This function mimics EXACTLY how imgs2csv.py calculates the value for the CSV
def get_mean_intensity(image_tensor):
    # Convert tensor (0-1) back to numpy (0-255)
    img_np = image_tensor.squeeze().cpu().detach().numpy()
    img_uint8 = (img_np * 255).astype(np.uint8)
    
    # Logic from provided imgs2csv.py
    image = img_uint8.astype(np.float32)
    if np.max(image) > 1.0: image = image / 255.0 
    min_val = np.min(image)
    max_val = np.max(image)
    
    if max_val - min_val > 0:
        image = (image - min_val) / (max_val - min_val)
    else:
        image = image - min_val 
        
    image_norm = np.uint8(image * 255.)
    return np.mean(image_norm)

# --- 4. DATASET CLASS ---
class DepthDataset(Dataset):
    def __init__(self, file_list, rgb_dir, depth_dir=None, is_test=False):
        self.file_list = file_list
        self.rgb_dir = rgb_dir
        self.depth_dir = depth_dir
        self.is_test = is_test

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_name = self.file_list[idx]
        rgb_path = os.path.join(self.rgb_dir, img_name)
        
        image = cv2.imread(rgb_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, IMG_SIZE)
        image = image / 255.0
        image = np.transpose(image, (2, 0, 1))
        
        sample = {'image': torch.tensor(image, dtype=torch.float32), 'name': img_name}

        if not self.is_test and self.depth_dir:
            depth_path = os.path.join(self.depth_dir, img_name)
            if not os.path.exists(depth_path):
                pre, ext = os.path.splitext(img_name)
                depth_path = os.path.join(self.depth_dir, pre + ".png")
            
            depth = cv2.imread(depth_path, cv2.IMREAD_GRAYSCALE)
            if depth is None:
                depth = np.zeros(IMG_SIZE, dtype=np.float32)
            else:
                depth = cv2.resize(depth, IMG_SIZE)
                depth = depth / 255.0
            
            depth = np.expand_dims(depth, axis=0)
            sample['depth'] = torch.tensor(depth, dtype=torch.float32)

        return sample

# --- 5. SPLIT DATA (TRAIN vs VALIDATION) ---
print("Splitting Data...")
all_files = sorted([f for f in os.listdir(TRAIN_RGB_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

# Use 85% for Training, 15% for Validation/Calculating Error
train_files, val_files = train_test_split(all_files, test_size=0.15, random_state=42)

train_dataset = DepthDataset(train_files, TRAIN_RGB_DIR, TRAIN_DEPTH_DIR)
val_dataset = DepthDataset(val_files, TRAIN_RGB_DIR, TRAIN_DEPTH_DIR)
test_dataset = DepthDataset(sorted(os.listdir(TEST_RGB_DIR)), TEST_RGB_DIR, is_test=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=2) # Batch 1 for accurate calc
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=2)

print(f"Training on {len(train_files)} images, Validating on {len(val_files)} images.")

# --- 6. MODEL SETUP ---
model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    activation='sigmoid'
).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.MSELoss() 

# --- 7. TRAINING WITH ERROR CALCULATION ---
best_val_rmse = float('inf')

for epoch in range(EPOCHS):
    # A. Training Step
    model.train()
    train_loss = 0.0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]"):
        images = batch['image'].to(DEVICE)
        depths = batch['depth'].to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, depths)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # B. Validation & Error Calculation Step
    model.eval()
    pred_means = []
    gt_means = []
    
    # We calculate the exact CSV metric (RMSE of Mean Intensities)
    with torch.no_grad():
        for batch in val_loader:
            images = batch['image'].to(DEVICE)
            depths = batch['depth'].to(DEVICE)
            
            preds = model(images)
            
            # Calculate mean intensity for Prediction and Ground Truth
            # (Using the helper function that copies imgs2csv.py logic)
            val_pred_mean = get_mean_intensity(preds[0])
            val_gt_mean = get_mean_intensity(depths[0])
            
            pred_means.append(val_pred_mean)
            gt_means.append(val_gt_mean)

    # Calculate RMSE between predicted means and actual means
    pred_means = np.array(pred_means)
    gt_means = np.array(gt_means)
    val_rmse = np.sqrt(np.mean((pred_means - gt_means)**2))

    print(f"Epoch {epoch+1} Results:")
    print(f"  > Train Pixel Loss: {train_loss/len(train_loader):.5f}")
    print(f"  > Validation RMSE (Score Estimate): {val_rmse:.5f}")

    # Save Best Model based on this RMSE score
    if val_rmse < best_val_rmse:
        best_val_rmse = val_rmse
        torch.save(model.state_dict(), 'best_model.pth')
        print(f"  --> New Best Score! Model Saved.")

# --- 8. FINAL INFERENCE ---
print(f"\nTraining Done. Best Validation RMSE was: {best_val_rmse:.5f}")
print("Generating Final Predictions...")
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Saving Images"):
        images = batch['image'].to(DEVICE)
        filenames = batch['name']
        preds = model(images)
        pred_depth = preds[0].squeeze().cpu().numpy()
        pred_depth_uint8 = (pred_depth * 255).astype(np.uint8)
        
        save_path = os.path.join(OUTPUT_DIR, filenames[0])
        if not save_path.endswith(('.jpg', '.png')): save_path += ".png"
        cv2.imwrite(save_path, pred_depth_uint8)

# --- 9. SUBMISSION CSV ---
def images_to_mean_csv(image_folder, output_csv):
    data = []
    print(f"\nProcessing submission file...")
    files = sorted([f for f in os.listdir(image_folder) if f.endswith(".png") or f.endswith(".jpg")])
    for filename in tqdm(files):
        filepath = os.path.join(image_folder, filename)
        image = cv2.imread(filepath, cv2.IMREAD_UNCHANGED)
        if image is None: continue
        
        # Exact logic replication
        image = image.astype(np.float32)
        if np.max(image) > 1.0: image = image / 255.0 
        min_val = np.min(image)
        max_val = np.max(image)
        if max_val - min_val > 0: image = (image - min_val) / (max_val - min_val)
        else: image = image - min_val 
        image_norm = np.uint8(image * 255.)
        mean_val = np.mean(image_norm)
        data.append([filename, mean_val])
    
    df = pd.DataFrame(data, columns=["id", "Predicted"])
    df.to_csv(output_csv, index=False)
    print("Done!")

images_to_mean_csv(OUTPUT_DIR, SUBMISSION_FILE)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour i

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Running on device: cuda
Splitting Data...
Training on 6800 images, Validating on 1200 images.


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

...

Epoch 15 Results:
  > Train Pixel Loss: 0.00380
  > Validation RMSE (Score Estimate): 18.39872

Training Done. Best Validation RMSE was: 17.33898
Generating Final Predictions...


Saving Images: 100%|██████████| 1000/1000 [00:10<00:00, 94.07it/s]



Processing submission file...


100%|██████████| 1000/1000 [00:00<00:00, 1482.99it/s]

Done!
